In [1]:
from pyspark.sql import SparkSession

In [2]:
spark_server = "spark://spark-master-otmzsp:7077"

In [3]:
# Configurando a sessão do Spark com as dependências e variáveis do S3A
# .master(spark_server) \
# .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
spark = SparkSession.builder \
    .appName("Processa Posições Raw") \
    .master(spark_server) \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio-otmzsp:9000") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [4]:
spark

In [5]:
raw = 's3a://raw/posicoes'
trusted = 's3a://trusted/posicoes'
checkpoint = 's3a://trusted/posicoes_checkpoint'

In [6]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType, TimestampType, FloatType

# Defining the schema for the JSON
schema = StructType([StructField("hr", StringType(), True),
                     StructField("l", ArrayType(StructType([StructField("c", StringType(), True),
                                                            StructField("cl", IntegerType(), True),
                                                            StructField("sl", IntegerType(), True),
                                                            StructField("lt0", StringType(), True),
                                                            StructField("lt1", StringType(), True),
                                                            StructField("qv", IntegerType(), True),
                                                            StructField("vs", ArrayType(StructType([StructField("p", IntegerType(), True),
                                                                                                    StructField("a", BooleanType(), True),
                                                                                                    StructField("ta", TimestampType(), True),
                                                                                                    StructField("py", FloatType(), True),
                                                                                                    StructField("px", FloatType(), True),
                                                                                                    StructField("sv", StringType(), True),
                                                                                                    StructField("is", StringType(), True)
                                                                                                    ])
                                                                                        )
                                                                        )
                                                            ])
                                                )
                                )
                    ])


In [7]:
# Leitura incremental de arquivos JSON da camada raw
df_stream = spark.read.format("json").schema(schema).load(raw + '/datepartition=2024-09-23')

In [8]:
df_stream.show()

+-----+--------------------+
|   hr|                   l|
+-----+--------------------+
|18:17|[{3686-10, 32934,...|
|18:19|[{3686-10, 32934,...|
|18:15|[{3686-10, 32934,...|
|18:09|[{3686-10, 32934,...|
|18:05|[{3686-10, 32934,...|
|18:01|[{3686-10, 32934,...|
|18:21|[{3686-10, 32934,...|
|18:11|[{3686-10, 32934,...|
|18:13|[{3686-10, 32934,...|
|17:57|[{3686-10, 32934,...|
|18:39|[{3686-10, 166, 1...|
|18:03|[{3686-10, 32934,...|
|17:59|[{3686-10, 32934,...|
|18:31|[{3686-10, 32934,...|
|18:23|[{3686-10, 32934,...|
|18:07|[{3686-10, 32934,...|
|18:25|[{3686-10, 32934,...|
|17:53|[{3686-10, 32934,...|
|18:33|[{3686-10, 32934,...|
|18:29|[{3686-10, 32934,...|
+-----+--------------------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import concat_ws, sha1
from pyspark.sql.functions import (
    explode, col, to_timestamp, when, from_utc_timestamp, to_date
)
from pyspark.sql.types import DoubleType

# Assumindo que 'df' é o DataFrame original carregado do JSON da SPTrans

df_final = df_stream.select(
    to_timestamp(col("hr")).alias("veiculo_horario_referencia"),
    explode("l").alias("linha")
).select(
    col("veiculo_horario_referencia"),
    col("linha.c").alias("veiculo_letreiro_completo"),
    col("linha.cl").alias("veiculo_linha_codigo"),
    col("linha.sl").alias("veiculo_sentido"),
    col("linha.lt0").alias("veiculo_letreiro_destino"),
    col("linha.lt1").alias("veiculo_letreiro_origem"),
    col("linha.qv").alias("qtde_veiculos_linha"),
    explode("linha.vs").alias("veiculo")
).select(
    "*",
    col("veiculo.p").alias("veiculo_prefixo"),
    col("veiculo.a").alias("veiculo_acessibilidade"),
    col("veiculo.ta").alias("veiculo_horario_utc_captura"),
    col("veiculo.py").cast(DoubleType()).alias("veiculo_latitude"),
    col("veiculo.px").cast(DoubleType()).alias("veiculo_longitude")
).drop("linha", "veiculo")

# Tratamento do sentido do veículo
df_final = df_final.withColumn(
    "veiculo_sentido",
    when(col("veiculo_sentido") == 1, "Bairro")
    .when(col("veiculo_sentido") == 2, "Centro")
    .otherwise(col("veiculo_sentido"))
)

# Tratamento da acessibilidade
df_final = df_final.withColumn(
    "veiculo_acessibilidade",
    when(col("veiculo_acessibilidade") == "true", "Acessível")
    .when(col("veiculo_acessibilidade") == "false", "Não acessível")
    .otherwise("Informação não disponível")
)

# Converter horário UTC para horário local (São Paulo, UTC-3)
df_final = df_final.withColumn(
    "veiculo_horario_local_captura",
    from_utc_timestamp(to_timestamp(col("veiculo_horario_utc_captura")), "America/Sao_Paulo")
)

# Adicionar coluna com o tipo de operação da linha
df_final = df_final.withColumn(
    "tipo_operacao_linha",
    when(col("veiculo_letreiro_completo").substr(-2, 2) == "10", "Linha Base")
    .when(col("veiculo_letreiro_completo").substr(-2, 2).isin("21", "23", "32", "41"), "Linha de Atendimento")
    .otherwise("Outro tipo de operação")
)

# Criar uma coluna 'id' com um identificador único usando SHA-256
df_final = df_final.withColumn(
    "id",
    sha1(
        concat_ws(
            "-", 
            col("veiculo_letreiro_completo"),
            col("veiculo_linha_codigo"),
            col("veiculo_prefixo"),
            col("veiculo_horario_local_captura"),
            col("veiculo_latitude"),
            col("veiculo_longitude")
        )
    )
)

In [10]:
df_final.show()

+--------------------------+-------------------------+--------------------+---------------+------------------------+-----------------------+-------------------+---------------+----------------------+---------------------------+-------------------+-------------------+-----------------------------+-------------------+--------------------+
|veiculo_horario_referencia|veiculo_letreiro_completo|veiculo_linha_codigo|veiculo_sentido|veiculo_letreiro_destino|veiculo_letreiro_origem|qtde_veiculos_linha|veiculo_prefixo|veiculo_acessibilidade|veiculo_horario_utc_captura|   veiculo_latitude|  veiculo_longitude|veiculo_horario_local_captura|tipo_operacao_linha|                  id|
+--------------------------+-------------------------+--------------------+---------------+------------------------+-----------------------+-------------------+---------------+----------------------+---------------------------+-------------------+-------------------+-----------------------------+-------------------+-----

In [11]:
df_final.count()
#1802572

3856862

In [12]:
from pyspark.sql import functions as F
import time

# Supondo que df_final já está definido e contém uma coluna 'id' e 'veiculo_horario_referencia'

# Remover registros duplicados com base no campo 'id'
df_final_unique = df_final.dropDuplicates(["id"])

# Adiciona uma coluna com o formato 'yyyy-MM-dd-HH' baseado em veiculo_horario_referencia
df_final_unique = df_final_unique.withColumn(
    "datepartition",
    F.date_format(F.col("veiculo_horario_local_captura"), "yyyy-MM-dd-HH")
)

# Ajuste o número de partições, se necessário
#df_final_unique = df_final_unique.repartition(4, "datepartition")

# Escreve o stream em CSV particionado por 'yyyy-MM-dd-HH', onde novos dados serão continuamente adicionados
df_final_unique.write \
    .format("csv") \
    .option("path", trusted) \
    .option("checkpointLocation", checkpoint) \
    .partitionBy("datepartition") \
    .option("header", "true") \
    .mode("append") \
    .save()

In [13]:
spark.stop()